# 0 导入必要的包

In [2]:
import numpy as np
import pandas as pd
import random
import sys
import os
from tqdm import tqdm
sys.path.append("../../tools/")
import commontools
import ucTools
import funclib
sys.path.append("../../")
import train
import time

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. 读取数据

In [2]:
# 1515 数据
sprot1515 = pd.read_csv('./data/ecoli1515/1515testset_with_unirep.tsv',  sep='\t')

In [3]:
sprot1515

,Unnamed: 0,id,ec_number,seq,isemzyme,isMultiFunctional,blattner_id,blattner_ec_number,0,1,...,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899
0,0,P42641,3.6.5.-,MKFVDEASILVVAGDGGNGCVSFRREKYIPKGGPDGGDGGDGGDVW...,True,False,b3183,NaN,0.003056,-0.087184,...,0.000332,-0.065542,0.117107,-0.236777,-0.622398,0.012113,0.010506,0.058984,0.061533,0.968014
1,1,P0AE22,3.1.3.2,MRKITQAISAVCLLFALNSSAVALASSPSPLNPGTNVARLAEQAPI...,True,False,b4055,3.1.3.2,0.001026,0.618393,...,0.030307,-0.244636,-0.038585,0.151263,-0.020389,0.095321,-0.031043,-0.881730,-0.196211,0.054455
2,2,P21499,3.1.13.1,MSQDPFQEREAEKYANPIPSREFILEHLTKREKPASRDELAVELHI...,True,False,b4179,3.1.-.-,0.073632,-0.003258,...,0.002123,0.060351,-0.008826,0.006141,-0.129352,-0.075010,0.023712,-0.002246,0.893310,0.140025
3,3,P39286,3.6.1.-,MSKNKLSKGQQRRVNANHQRRLKTSKEKPDYDDNLFGEPDEGIVIS...,True,False,b4161,3.6.1.-,0.000775,-0.008305,...,0.066885,0.020991,0.073577,-0.034962,-0.036755,0.053803,-0.017355,0.100456,-0.202344,0.351100
4,4,P56100,7.1.1.7,MWYFAWILGTLLACSFGVITALALEHVESGKAGQEDI,True,False,b4515,1.10.3.10,0.001811,-0.050181,...,0.042016,-0.028950,0.305668,-0.105587,-0.113769,0.680413,0.138797,-0.268528,-0.076093,-0.038212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4745,4745,P36837,NaN,MNTTTPMGMLQQPRPFFMIFFVELWERFGYYGVQGVLAVFFVKQLG...,False,False,b3496,NaN,0.000070,-0.021424,...,0.005911,-0.016855,0.377301,-0.500727,-0.247861,0.752678,-0.005501,0.396191,-0.215832,-0.062812
4746,4746,P77304,NaN,MSTANQKPTESVSLNAFKQPKAFYLIFSIELWERFGYYGLQGIMAV...,False,False,b1634,NaN,0.008628,-0.065382,...,-0.001197,-0.101824,0.013007,-0.304450,-0.295525,-0.124492,0.010846,0.032853,-0.155702,-0.183535
4747,4747,P27248,2.1.2.10,MAQQTPLYEQHTLCGARMVDFHGWMMPLHYGSQIDEHHAVRTDAGM...,True,False,b2905,2.1.2.10,0.006723,0.227349,...,-0.124573,-0.032099,-0.170996,0.104425,-0.163041,0.183720,-0.038814,0.036993,-0.044558,0.010447
4748,4748,P0AC88,4.2.1.47,MSKVALITGVTGQDGSYLAEFLLEKGYEVHGIKRRASSFNTERVDH...,True,False,b2053,4.2.1.47,0.000665,-0.029179,...,0.145032,0.867074,-0.263008,0.009123,0.036783,-0.073805,0.439632,0.014501,-0.205913,-0.005663


In [4]:
funclib.table2fasta(sprot1515, './data/sprot1515.fasta')

Write finished


## 2. 序列比对

In [5]:
train = pd.read_hdf('./data/train.h5',key='data')
test = pd.read_hdf('./data/test.h5',key='data')
head = funclib.table_head + ['f'+str(i) for i in range(1, 1901) ]
head = head + ['ec_label','ec_appears']
train.columns = head
test.columns = head

In [ ]:
train = pd.read_hdf('./data/train.h5',key='data')

In [4]:
file_enzyme_train_x = '../../data/preprocess/enzyme_train_x.feather'
file_enzyme_train_y = '../../data/preprocess/enzyme_train_y.feather'
DATE_TRAIN ='2018-01-01'
uctools =  ucTools.ucTools('172.16.25.20')
cnx_ecnumber = uctools.db_conn()

eclablefile='../../data/dict_ec_label.npy'
featurefile='../../data/sprot_unirep.feather'

dict_ec_label, dict_label_ec = train.load_ec_lable_dict(eclablefile)

In [6]:
len(dict_ec_label)

5389

In [59]:
enzyme_X, enzyme_Y = train.get_enzyme_train_set( train_date=DATE_TRAIN, 
                            eclablefile='../../data/dict_ec_label.npy', 
                            featurefile='../../data/sprot_unirep.feather', 
                            cnx=cnx_ecnumber,
                            trainX=file_enzyme_train_x,
                            trainY=file_enzyme_train_y
                          )

In [60]:
file_isenzyme_model = './model/isenzyme.model'
train.train_isenzyme(enzyme_X, enzyme_Y.enzyme_label, model_file=file_isenzyme_model)

XGBoost模型训练完成


In [ ]:
eclablefile='../../data/dict_ec_label.npy'
featurefile='../../data/sprot_unirep.feather'

dict_ec_label, dict_label_ec = train.load_ec_lable_dict(eclablefile)
train_data = train.load_train_file(split_date= DATE_TRAIN, cnx_ecnumber=cnx_ecnumber, type='ec')

#添加训练标签
train_data_labeled  = train.add_ec_label(train_data, dict_ec_label)

unirep_data = train.load_unirep(featurefile)

res_data = train_data_labeled.merge(unirep_data, on='id', how='left')

In [7]:
enzyme_X, enzyme_Y = train.get_enzyme_train_set( train_date=DATE_TRAIN, 
                            eclablefile='./data/dict_ec_label.npy', 
                            featurefile='./data/sprot_unirep.feather', 
                            cnx=cnx_ecnumber,
                            trainX=file_enzyme_train_x,
                            trainY=file_enzyme_train_y
                          )

In [29]:

file_ec_train_x = '../../data/preprocess/ec_train_x.feather'
file_ec_train_y = '../../data/preprocess/ec_train_y.feather'
ec_X, ec_Y = train.get_ec_train_set(train_date=DATE_TRAIN,
                                eclabelfile='../../data/dict_ec_label.npy',
                                featurefile='../../data/sprot_unirep.feather',
                                cnx=cnx_ecnumber,
                                trainX=file_ec_train_x,
                                trainY=file_ec_train_y
                            )

In [32]:
max(ec_Y.ec_label)

5388

In [34]:
len(set(ec_Y.ec_label))

4891

In [36]:
for i in range(3):
    print(i)

0
1
2


In [41]:
id_ec_dict = {v: k for v,k in zip( set(ec_Y.ec_label), range(len(set(ec_Y.ec_label))))}

In [47]:
id_ec_dict.get(5000)

4548

In [23]:
train_Y = pd.read_feather('../../data/preprocess/ec_train_y.feather')

In [28]:
train_Y

,id,seq,ec_label
0,P75471,MNDTDKKFPLQPVYDTGFDDGYLQRDYEKCLESAAANDAQTVELQT...,0
1,Q9KSU0,MRSHYCGHLNKSLVGQTVELCGWVNRRRDLGGLIFIDMRDREGIVQ...,5108
2,Q09219,MTNSAMALPPTRINRRRSTKALHDFFSRPFTQRRSQSIAGVRVSPR...,0
3,P56159,MFLATLYFALPLLDLLLSAEVSGGDRLDCVKASDQCLKEQSCSTKY...,0
4,P80954,GLLNTFKDWAISIAKGAGKGVLTTLSCKLDKSC,0
...,...,...,...
578273,Q62953,YCYICHSLQYNADLQPANTCIYLVVTWVMTVLDVLPNVYIGTIEYD...,0
578274,O08601,MILLAVLFLCFFSSYSASVKGHTTGLSLNNERLYKLTYSTEVFLDG...,0
578275,Q91955,MSDKEFMWALKNGDLDEVKDYVAKGEDVNRTLEGGRKPLHYAADCG...,0
578276,B7MAE6,MSDYKSTLNLPETGFPMRGDLAKREPGMLARWTDDDLYGIIRAAKK...,5124


In [22]:
file_slice_train_x ='../../data/preprocess/ec_slice_train_x.txt'
file_slice_train_y ='../../data/preprocess/ec_slice_train_y.txt'

train.prepare_slice_file(x_data=ec_X,y_data=ec_Y['ec_label'],x_file=file_slice_train_x, y_file=file_slice_train_y)